In [1]:
pip install transformers datasets evaluate seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16164 sha256=2d169d3d1df1bc4f1650dc749b0a17b567d7022fc981ba68a337cda5770ad47a
  Stored in directory: /Users/nm/Library/Caches/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
Note: you may need to restart the kernel to use updated packages.


# Task

Token classification assigns a label to individual tokens in a sentence. One of the most common token classification tasks is Named Entity Recognition (NER). NER attempts to find a label for each entity in a sentence, such as a person, location, or organization.

This notebook shows how to:
1. Finetune DistilBERT on the WNUT 17 dataset to detect new entities.
2. Use the finetuned model for inference.

# Libraries

In [2]:
from datasets import load_dataset

# Data

In [3]:
# Load the WNUT 17 dataset from the 🤗 Datasets library
wnut = load_dataset("wnut_17")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/3394 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1009 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1287 [00:00<?, ? examples/s]

In [4]:
# Check out an example
wnut["train"][0]

{'id': '0',
 'tokens': ['@paulwalk',
  'It',
  "'s",
  'the',
  'view',
  'from',
  'where',
  'I',
  "'m",
  'living',
  'for',
  'two',
  'weeks',
  '.',
  'Empire',
  'State',
  'Building',
  '=',
  'ESB',
  '.',
  'Pretty',
  'bad',
  'storm',
  'here',
  'last',
  'evening',
  '.'],
 'ner_tags': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  8,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

In [8]:
tokens_list = wnut["train"][0]['tokens']
ner_tags_list = wnut["train"][0]['ner_tags']
print(tuple(zip(tokens_list, ner_tags_list)))

(('@paulwalk', 0), ('It', 0), ("'s", 0), ('the', 0), ('view', 0), ('from', 0), ('where', 0), ('I', 0), ("'m", 0), ('living', 0), ('for', 0), ('two', 0), ('weeks', 0), ('.', 0), ('Empire', 7), ('State', 8), ('Building', 8), ('=', 0), ('ESB', 7), ('.', 0), ('Pretty', 0), ('bad', 0), ('storm', 0), ('here', 0), ('last', 0), ('evening', 0), ('.', 0))


In [5]:
# Each number in ner_tags represents an entity
# Convert the numbers to their label names to find out what the entities are
# NB: The letter that prefixes each ner_tag indicates the token position of the entity:
# B - indicates the beginning of an entity.
# I - token is contained inside the same entity (e.g. State token is a part of an entity like Empire State Building).
# 0 indicates the token doesn’t correspond to any entity
label_list = wnut["train"].features[f"ner_tags"].feature.names
label_list

['O',
 'B-corporation',
 'I-corporation',
 'B-creative-work',
 'I-creative-work',
 'B-group',
 'I-group',
 'B-location',
 'I-location',
 'B-person',
 'I-person',
 'B-product',
 'I-product']